# Installations

# Import statements

# Global variables

# Functions

# Program start

## Preliminaries

### Mount Google Drive

### Retrieve stereo images

### Display stereo images

## Camera Calibration, and Image Rectification

## Image Segmentation

### Segment plant objects

### Save segmented images

### Display segmented images

## Depth Map Generation

### Generate depth maps

### Save depth maps

### Display depth maps

## Point Cloud Generation

### Generate point clouds

### Save point clouds

### Display point clouds

## Point Cloud Registration

### Merge point clouds

### Save merged point cloud

### Display merged point cloud

## Mesh Generation

### Generate mesh from merged point cloud

### Save 3D mesh model

### Display 3D mesh model